In [1]:
import sys
import os

# Add src to the system path to allow imports from the src directory
notebook_dir = os.getcwd()
src_path = os.path.abspath(os.path.join(notebook_dir, '../../..', 'src'))
sys.path.insert(0, src_path)

In [2]:
from dolfin import *
import ufl
from materials import *
from params import *
from matplotlib import pyplot as plt
%matplotlib inline

/opt/anaconda3/envs/fenicsproject/lib/python3.10/site-packages/dolfin/jit/jit.py:121: RuntimeWarning: mpi4py.MPI.Session size changed, may indicate binary incompatibility. Expected 32 from C header, got 40 from PyObject
  def compile_class(cpp_data, mpi_comm=MPI.comm_world):


## Initialize problem

In [3]:
params = Params()
params.print_table()

| Parameter    |        Value | Unit   |
|--------------|--------------|--------|
| dimension    |      3       | -      |
| plane_stress |      0       | Bool   |
| Lx           |    500       | m      |
| Ly           |    750       | m      |
| Lz           |    125       | m      |
| g            |      9.81    | m/s²   |
| rho_sea      |  10006.2     | N/m³   |
| E0_ice       |      9.5e+09 | Pa     |
| nu_ice       |      0.35    | -      |
| rho_i_ice    |   8995.77    | N/m³   |
| sigma_c_ice  | 474000       | Pa     |
| ci           |      1       | -      |


In [4]:
g = params.g
rho_sea = params.rho_sea
E0_ice = params.E0_ice
nu_ice = params.nu_ice
rho_i_ice = params.rho_i_ice
sigma_c_ice = params.sigma_c_ice

In [5]:
xdmf = XDMFFile("output/static/no_crevasse_3D.xdmf")
xdmf.parameters["functions_share_mesh"] = True
xdmf.parameters["rewrite_function_mesh"] = False
xdmf.parameters["flush_output"] = True

## Initialize Mesh

In [6]:
Lx, Ly, Lz = 500, 750, 125
mul = 1/5
nx, ny, nz = int(Lx*mul), int(Ly*mul), int(Lz*mul)
mesh = BoxMesh(Point(0, 0, 0), Point(Lx, Ly, Lz), nx, ny, nz)

## Initialize material model

In [7]:
ice = LinearElastic(mesh, params)

# Initialize boundary conditions

In [8]:
FDG0 = FunctionSpace(mesh, "DG", 0)
TDG0 = TensorFunctionSpace(mesh, "DG", 0)
V1 = VectorFunctionSpace(mesh, "CG", 1)

/opt/anaconda3/envs/fenicsproject/lib/python3.10/site-packages/dolfin/jit/jit.py:46: RuntimeWarning: mpi4py.MPI.Session size changed, may indicate binary incompatibility. Expected 32 from C header, got 40 from PyObject
  if MPI.size(mpi_comm) == 1:


In [9]:
front = CompiledSubDomain("near(x[1], 0)")
back = CompiledSubDomain("near(x[1], L)",L=Ly)
left = CompiledSubDomain("near(x[0], 0)")
right_csd = CompiledSubDomain("near(x[0], B)", B=Lx)
bottom = CompiledSubDomain("near(x[2], 0)")
top = CompiledSubDomain("near(x[2], H)", H=Lz)

bottom_roller = DirichletBC(V1.sub(2), Constant(0), bottom)
left_roller = DirichletBC(V1.sub(0), Constant(0), left)
front_roller = DirichletBC(V1.sub(1), Constant(0), front)
back_roller = DirichletBC(V1.sub(1), Constant(0), back)
bc_u = [bottom_roller, left_roller,front_roller,back_roller]

boundaries = MeshFunction("size_t", mesh, mesh.topology().dim() - 1)
right_csd.mark(boundaries, 1)

ds = Measure("ds", subdomain_data=boundaries)

## Setup and Solve Problem

In [10]:
u, v = TrialFunction(V1), TestFunction(V1)
u_sol = Function(V1, name="disp")
stress_xx, cdf = Function(FDG0, name="sigma_xx"), Function(FDG0, name="cdf")

In [11]:
# t = Expression(("-(rho_sea*(125/2 -x[1]))", 0), rho_sea=rho_sea, degree=1)

t = Constant((0, 0,0))
f = Constant((0, 0,-ice.rho))

In [12]:
a = inner(ice.stress(u), ice.strain(v)) * dx
L = dot(f, v) * dx + dot(t, v) * ds(1)

In [13]:
solve(a == L, u_sol, bc_u, solver_parameters={"linear_solver": "gmres", "preconditioner": "ilu"})

Solving linear variational problem.


## Post Processing

In [15]:
stress_xx.assign(project(ice.principal_stress(u_sol)[0,0]/1e6, FDG0)) # N/m2 -> MPa
cdf.assign(project(ice.crack_driving_force(u_sol), FDG0))

Calling FFC just-in-time (JIT) compiler, this may take some time.


In [18]:
xdmf.write(u_sol,0)
xdmf.write(stress_xx,0)
xdmf.write(cdf,0)
xdmf.close()

In [19]:
cdf.vector().max()

0.0

In [20]:
stress_xx.vector().max()

0.2962848843719384